## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Vao,NC,-22.6667,167.4833,70.23,65,99,15.88,overcast clouds
1,1,Carnarvon,AU,-24.8667,113.6333,65.59,64,0,11.63,clear sky
2,2,Lamu,KE,-2.2717,40.9020,79.02,77,1,18.16,light rain
3,3,Thompson,CA,55.7435,-97.8558,60.96,88,100,10.36,overcast clouds
4,4,Kodinsk,RU,58.6881,99.1844,55.44,89,9,2.91,clear sky


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Lamu,KE,-2.2717,40.9020,79.02,77,1,18.16,light rain
6,6,Daru,PG,-9.0763,143.2092,76.95,90,100,17.78,moderate rain
11,11,Avarua,CK,-21.2078,-159.7750,82.45,78,75,10.36,broken clouds
17,17,Albany,US,42.6001,-73.9662,79.63,36,51,1.01,broken clouds
19,19,Hirara,JP,24.8000,125.2833,82.67,89,40,10.36,scattered clouds
23,23,Hithadhoo,MV,-0.6000,73.0833,83.57,71,33,11.45,scattered clouds
27,27,Buala,SB,-8.1450,159.5921,82.33,74,86,3.27,overcast clouds
32,32,Isangel,VU,-19.5500,169.2667,75.31,73,20,4.61,few clouds
34,34,Banda Aceh,ID,5.5577,95.3222,78.21,86,98,10.29,moderate rain
36,36,Sola,VU,-13.8833,167.5500,80.20,74,27,12.82,scattered clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

Unnamed: 0             216
City                   216
Country                216
Lat                    216
Lng                    216
Max Temp               216
Humidity               216
Cloudiness             216
Wind Speed             216
Current Description    216
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna(subset = ['Country'])
clean_df.reset_index(drop=True)

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,2,Lamu,KE,-2.2717,40.9020,79.02,77,1,18.16,light rain
1,6,Daru,PG,-9.0763,143.2092,76.95,90,100,17.78,moderate rain
2,11,Avarua,CK,-21.2078,-159.7750,82.45,78,75,10.36,broken clouds
3,17,Albany,US,42.6001,-73.9662,79.63,36,51,1.01,broken clouds
4,19,Hirara,JP,24.8000,125.2833,82.67,89,40,10.36,scattered clouds
...,...,...,...,...,...,...,...,...,...,...
211,692,Robore,BO,-18.3333,-59.7500,83.39,38,5,7.78,clear sky
212,693,Cam Ranh,VN,11.9214,109.1591,78.76,78,0,2.30,clear sky
213,700,Yuzhou,CN,34.1628,113.4639,82.60,50,98,4.41,overcast clouds
214,701,Lhokseumawe,ID,5.1801,97.1507,77.00,86,93,4.16,overcast clouds


In [7]:
# Determine if there are any empty rows.
clean_df.count()

Unnamed: 0             216
City                   216
Country                216
Lat                    216
Lng                    216
Max Temp               216
Humidity               216
Cloudiness             216
Wind Speed             216
Current Description    216
dtype: int64

In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = None
hotel_df.head(10)
hotel_df.count()

City                   216
Country                216
Max Temp               216
Current Description    216
Lat                    216
Lng                    216
Hotel Name               0
dtype: int64

In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]
    
    # 6c. Get latitude and longitude from DataFrame
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
   
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except(IndexError):
        print("Hotel not found...skipping.")
        

Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.


In [10]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna(subset = ['Hotel Name'])
clean_hotel_df.reset_index(drop=True)
clean_hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Lamu,KE,79.02,light rain,-2.2717,40.9020,Lamu House
6,Daru,PG,76.95,moderate rain,-9.0763,143.2092,Daru Lodge
11,Avarua,CK,82.45,broken clouds,-21.2078,-159.7750,Paradise Inn
19,Hirara,JP,82.67,scattered clouds,24.8000,125.2833,Hotel Atoll Emerald Miyakojima
23,Hithadhoo,MV,83.57,scattered clouds,-0.6000,73.0833,Scoop Guest House


In [11]:
# 8a. Create the output File (CSV)
city_data_df.to_csv('WeatherPy_Vacation.csv')
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv('WeatherPy_Vacation.csv', index_label="City_ID")

In [12]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description}</dd>
</dl>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [13]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))